In [ ]:
# Version History
#print("Version 1.0.0: 09/23/2022 5:06pm - Nate Calvanese - First version created")
#print("Version 1.0.1: 09/26/2022 11:18m - Nate Calvanese - Fixed bug in default dataset naming")
#print("Version 1.0.2: 09/27/2022 2:43pm - Nate Calvanese - Added ability to aggregate multiple workspaces into one dataset")
#print("Version 1.0.3: 10/5/2022 1:32pm - Nate Calvanese - Added support for chunking up ingest requests")
#print("Version 1.0.4: 10/6/2022 10:35am - Nate Calvanese - Updated use of TDR utility functions")
print("Version 1.0.5: 10/13/2022 10:54am - Nate Calvanese - Parameter tweaks for latest changes")


# Imports and Common Variables

In [ ]:
# Install additional modules (one time effort per cloud environment)
!pip install --upgrade import_ipynb data_repo_client
#!pip install data_repo_client==1.409.0

In [ ]:
# Workspace environment variables
import os
import re
print("Recording workspace environment variables:")
ws_name = os.environ["WORKSPACE_NAME"]
ws_project = os.environ["WORKSPACE_NAMESPACE"]
ws_bucket = os.environ["WORKSPACE_BUCKET"]
ws_bucket_name = re.sub('^gs://', '', ws_bucket)
print(f"Workspace name = {ws_name}")
print(f"Workspace project = {ws_project}")
print(f"Workspace bucket = {ws_bucket}")
print(f"Workspace bucket name = {ws_bucket_name}")

# Copy latest version of the pipeline notebooks to the cloud environment (uncomment if any notebooks have changed since last run)
# print("\nCopying latest pipeline notebooks to the cloud environment:")
# !gsutil -m cp $ws_bucket/notebooks/*.ipynb .

# Additional imports
print("\nRunning imports:")
import import_ipynb
import pandas as pd
from firecloud import api as fapi
import data_repo_client
import ingest_pipeline_utilities as utils
import build_mapping_query as bmq
from google.cloud import storage
from google.cloud import bigquery
import google.auth
import google.auth.transport.requests
import logging
import datetime
import json

# Common pipeline variables (AnVIL)
ws_attributes = utils.get_workspace_attributes(ws_project, ws_name)
params = {}
params["ws_name"] = ws_name
params["ws_project"] = ws_project
params["ws_bucket"] = ws_bucket
params["ws_bucket_name"] = ws_bucket_name
params["profile_id"] = "e0e03e48-5b96-45ec-baa4-8cc1ebf74c61"
params["google_project"] = ws_attributes["googleProject"]
params["create_file_table"] = True
params["file_table_name"] = "file_inventory"
params["anvil_schema_version"] = "ANV2"

# Configure logging format
logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p", level=logging.INFO)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)


# "EL" Pipeline: Load Dataset to TDR in Source Format

## Pipeline Run Variables

In [ ]:
## >>> Run Variables <<<
# For datasets split across multiple workspaces, set the staging area and target TDR dataset to the 
# same value to collect all of the source data and process it together.
workspace_run_list = [
    #["Workspace_Name", "Workspace_Project", "Staging Area (Leave empty for default)", "Target_TDR_Dataset_Name (Leave empty for default)", Run (True/False)]
    ["ANVIL_1234", "anvil-datastorage", "", "", False],
    ["AnVIL_5678", "anvil-datastorage", "", "", False]
]
params["skip_source_files_creation"] = False
params["skip_file_inventory_creation"] = False
params["skip_table_data_processing"] = False
params["skip_ingests"] = False
params["skip_snapshot_creation"] = False
params["snapshot_readers_list"] = ["anvil_tdr_ingest@firecloud.org", "azul-anvil-prod@firecloud.org"] # Auth domain groups are also added as readers automatically


## >>> File Inventory Variables <<<
# The GCS bucket associated with the source workspace will be automatically included in the file inventory build. To specify 
# additional GCS buckets to include in the file inventory build, add entries to the below dictionary.
params["additional_file_inventory_sources"] = {}
# EXAMPLE:
# params["additional_file_inventory_sources"] = {
#     "staging_area": {
#         "bucket_name": {
#             "include_dirs": [], # Leave empty to include all directories in bucket
#             "exclude_dirs": [] # Exclusions will take precedence over inclusions
#         }
#     }
# }


## >>> Ingest Variables <<<
# For cases where you only want to ingest a subset of files, use the below dictionary to specify exactly what should be ingested.
params["ingest_list_override"] = {}
# EXAMPLE:
# params["ingest_list_override"] = {
#     "ws_table": ["ws_table_0.json"], # Leave empty to run ingest for every file for target table
# }


## >>> File Reference Variables <<<
# Fields containing GCS links will be identified automatically by the pipeline. The below dict should contain any fields
# that contain file references that aren't proper GCS links in the workspace tables.
data_file_refs_dict = {   
    "sequencing": [{
        "column": "sequencing_id",
        "method": "file_path_match",
        "match_multiple_files": False, 
        "match_regex": None,
        "mode": "fileref_in_line",
        "create_new_field": True,
        "new_field_name": "sequencing_id_fileref"
    }]
}
# Definitions:
#    Required Fields: column, method, mode, create_new field
#    Optional Fields: match_multiple_files (default to True), match_regex (default to None), new_field_name (default to None)
#    Methods: 
#       file_path_match -- Field contains a full or partial file path, which can be matched to the file inventory to grab the file(s) referenced 
#       tdr_file_id -- Field contains file UUIDs of files already ingested into the target TDR dataset
#    Modes:
#       fileref_in_line -- Populates the field with a file reference object
#       fileref_table_ref -- Populates the field with an ID that joins to a file table. If no file table built, falls back on fileref_in_line logic.
    
#-----------------------------------------------------------------------------------------------------------#
    
# Print variables
print("Pipeline run variables set:")
print("Profile ID: " + params["profile_id"])
print("Ingests to run: ")
current_datetime = datetime.datetime.now()
current_date_string = current_datetime.strftime("%Y%m%d")
for workspace in workspace_run_list:
    if workspace[4] == True:
        ws_attributes = utils.get_workspace_attributes(workspace[1], workspace[0])
        params["phs_id"] = utils.format_phs_id(ws_attributes["attributes"]["phs_id"]) if ws_attributes["attributes"].get("phs_id") else ""
        auth_list = ws_attributes["authorizationDomain"] if ws_attributes.get("authorizationDomain") else []
        params["auth_domains"] = [x["membersGroupName"] for x in auth_list]
        params["consent_name"] = ws_attributes["attributes"]["library:dataUseRestriction"] if ws_attributes["attributes"].get("library:dataUseRestriction") else ""
        params["data_files_src_bucket"] = ws_attributes["bucketName"] if ws_attributes.get("bucketName") else ""
        workspace[3] = workspace[3] if workspace[3] else utils.format_dataset_name(workspace[0])
        workspace[2] = workspace[2] if workspace[2] else workspace[0]
        print("- Workspace [" + workspace[1] + "/" + workspace[0] + "] to TDR dataset [" + workspace[3] + "] via Staging Area [" + workspace[2] + "]")
        print("\t- PHS ID = " + params["phs_id"])
        print("\t- Consent Short Name = " + params["consent_name"])
        print("\t- Auth Domains = " + str(params["auth_domains"]))
        print("\t- Data Files Source Bucket = " + params["data_files_src_bucket"])
print("Skip source files creation? " + str(params["skip_source_files_creation"]))
print("Skip file inventory creation? " + str(params["skip_file_inventory_creation"]))
print("Skip table data processing? " + str(params["skip_table_data_processing"]))
print("Skip ingests? " + str(params["skip_ingests"]))
print("Skip snapshot creation? " + str(params["skip_snapshot_creation"]))


## Pipeline Execution

In [ ]:
# Loop through and execute workspace connector pipeline ("E") for listed workspaces
if params["skip_source_files_creation"] == True:
    logging.info("Skipping source file creation, per user request.")
else:
    for workspace in workspace_run_list:
        if workspace[4] == True:
            params["data_file_refs"] = data_file_refs_dict  
            utils.run_ws_connector_pipeline(workspace, params)

# Aggregate staging area to target dataset combinations, loop through them, and execute ingest pipeline ("L")
pipeline_run_list = []
for workspace in workspace_run_list:
    if workspace[4] == True:
        temp_list = [workspace[2], workspace[3]]
        if temp_list not in pipeline_run_list:
            pipeline_run_list.append(temp_list)
for pipeline in pipeline_run_list:
    utils.run_el_pipeline(pipeline, params)


# Mapping Development
Work through the following steps for each dataset that needs to be processed through the transformation pipeline in Step 4. Note that you can use the logs or results_dict from the previous step to retrieve the dataset_id values of interest, or retrieve them directly from TDR via the UI or Swagger.

In [ ]:
# Set the dataset for mapping development
dataset_id = "1234"
print(dataset_id)

## Add Missing Relationships to TDR Dataset Schema
Relationships are needed by the mapping query constructor to build appropriate joins between tables. If no joins are required between tables, this step is unnecessary. 

In [ ]:
# Record relationships to potentially add to source dataset. Note that there may be more relationships to add
# than those listed below, so add to this list as necessary.
potential_relationships = [
#     ["subject.family_id", "family.family_id"],
#     ["sample.subject_id", "subject.subject_id"],
#     ["sequencing.sample_id", "sample.sample_id"],
#     ["sample.participant", "participant.participant_id"],
]

# Retrieve source schema
api_client = utils.refresh_tdr_api_client()
datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
src_schema_dict = {}
try:
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    response = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION"]).to_dict()
    src_schema_dict["tables"] = response["schema"]["tables"]
    src_schema_dict["relationships"] = response["schema"]["relationships"]
except Exception as e:
    print("Error retrieving source schema from TDR. Error: {}".format(e))

# Loop through potential relationships and add those present for the source dataset
additional_relationships = []
for rel in potential_relationships:
    from_table = rel[0].split(".")[0] 
    from_column = rel[0].split(".")[1]
    to_table = rel[1].split(".")[0]
    to_column = rel[1].split(".")[1]
    if bmq.confirm_column_exists(src_schema_dict, from_table, from_column) and bmq.confirm_column_exists(src_schema_dict, to_table, to_column):
        relationship_found = False
        for rel_entry in src_schema_dict["relationships"]:
            if rel_entry["_from"]["table"] == from_table and rel_entry["_from"]["column"] == from_column and rel_entry["to"]["table"] == to_table and rel_entry["to"]["column"] == to_column:
                relationship_found = True
            elif rel_entry["_from"]["table"] == to_table and rel_entry["_from"]["column"] == to_column and rel_entry["to"]["table"] == from_table and rel_entry["to"]["column"] == from_column:
                relationship_found = True
        if not relationship_found:
            rel_dict = {
                "name": from_table + "_" + from_column + "__to__" + to_table + "_" + to_column,
                "from": {"table": from_table, "column": from_column},
                "to": {"table": to_table, "column": to_column}
            }
            additional_relationships.append(rel_dict)

# Submit the schema update request for the TDR dataset
if additional_relationships:
    schema_update_request = {
        "description": "Adding relationships to support query construction.",
        "changes": {
            "addRelationships": additional_relationships
        }
    }
    try:
        resp = utils.wait_for_tdr_job(datasets_api.update_schema(id=dataset_id, dataset_schema_update_model=schema_update_request))
        print(str(resp)[0:1000])
    except Exception as e:
        print("Error running schema update: " + str(e))
else:
    print("No additional relationships to add to schema.")

## Retrieve Mapping Artifacts and Run Query Construction
Specify the target schema ("mapping target") and mapping specification ("mapping_target_spec") you would like to use to construct transformation queries for your dataset. These transformation queries will then be dynamically constructed based on the appropriate target schema, mapping specification, and source schema. 

In [ ]:
# Set the desired mapping specification and retrieve artifacts needed for query construction
mapping_target = "anvil"
mapping_target_spec = "cmg_ext_1" 

# Retrieve source schema
api_client = utils.refresh_tdr_api_client()
datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
src_schema_dict = {}
try:
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    response = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION"]).to_dict()
    src_schema_dict["name"] = response["name"]
    src_schema_dict["tables"] = response["schema"]["tables"]
    src_schema_dict["relationships"] = response["schema"]["relationships"]
    bq_project = response["access_information"]["big_query"]["project_id"]
    bq_schema = response["access_information"]["big_query"]["dataset_name"]
except Exception as e:
    print("Error retrieving source schema from TDR. Error: {}".format(e))
#print(src_schema_dict)

# Set dataset_name parameter to substitute into transform queries ($DATASET_NAME)
dataset_name_value = re.sub("(_[0-9]+$)", "", src_schema_dict["name"])

# Retrieve target schema and mapping specification
target_schema_dict = {}
mapping_spec = {}
storage_client = storage.Client()
bucket = storage_client.get_bucket(ws_bucket_name)
try:
    blob = bucket.blob(f"ingest_pipeline/mapping/{mapping_target}/mapping_schema_object.json")
    target_schema_dict = json.loads(blob.download_as_string(client=None))
except Exception as e:
    print("Error retrieving target schema for specified mapping_target. Error: {}".format(e))
#print(target_schema_dict)
try:
    blob = bucket.blob(f"ingest_pipeline/mapping/{mapping_target}/{mapping_target_spec}/mapping_specification.json")
    blob_string = blob.download_as_text(client=None)
    blob_string = blob_string.replace("$DATASET_NAME", dataset_name_value)
    mapping_spec = json.loads(blob_string)
except Exception as e:
    print("Error retrieving mapping specification for specified mapping_target and mapping_target_spec. Error: {}".format(e))
#print(mapping_spec)

# Build queries from mapping specification
query_dict = {}
if target_schema_dict:
    for target_table in target_schema_dict["tables"]:
        table_name = target_table["name"]
        missing_artifacts = False
        if src_schema_dict and mapping_spec:
            query_dict[table_name] = bmq.build_mapping_query(target_table, src_schema_dict, mapping_spec, bq_project, bq_schema)
        else:
            missing_artifacts = True
            query_dict[table_name] = {"query": "", "syntax_check": ""} 
    if missing_artifacts == True:
        print("Source schema dictionary and/or mapping specification missing. Unable to generate queries.")
else:
    print("Target schema dictionary missing. Unable to generate queries.")
query_df = pd.DataFrame.from_dict(query_dict, orient="index")
query_df.index.name = "target_table"
query_df.reset_index(inplace=True)
display(query_df)
    

## [Optional] Review Generated Queries and Update/Override as Necessary
Review the queries generated in the previous step. Any queries that have not passed the syntax check need to be remedied before they can be executed. Any other queries that do not align with expectations can be overridden by either A) Updating the mapping target specification, or B) Manually overriding the query below. Option B should only ideally only be used in one-off cases.

In [ ]:
# To update the query definition for particular target table, input the target table and query below
target_table = ""
query = ""

# Run syntax check
query_dict[target_table]["query"] = query
query_dict[target_table]["syntax_check"] = bmq.run_syntax_check(query)
print(query_dict[target_table])


## Evaluate Vocabulary Mapping
For target attributes leveraging the "VOCAB_MAP" transformation, evaluate whether the source values have a record in the dsp-data-ingest.transform_resources.vocab_map table. If additional mappings are needed, these should be put into place before the transformation queries are executed.

In [ ]:
# Evaluate vocab mapping and display results
df = bmq.evaluate_vocab_mapping(mapping_spec, src_schema_dict, target_schema_dict, bq_project, bq_schema)
print("-------------------------------------------")
print("Missing mapped_value view:")
print("-------------------------------------------")
display(df[df["mapped_value"].isnull() & df["source_value"].notnull()])
print("\n-------------------------------------------")
print("Full view:")
print("-------------------------------------------")
display(df)

## Publish Mapping Artifacts for use in "T" Pipeline

In [ ]:
# Copy target schema file to output folder for mapping target
source_path = "ingest_pipeline/mapping/{}/mapping_schema_object.json".format(mapping_target)
destination_path = "ingest_pipeline/output/transformed/{}/{}/schema/mapping_schema_object.json".format(mapping_target, dataset_id)
!gsutil cp $ws_bucket/$source_path $ws_bucket/$destination_path 2> stdout

# Limit query dict to valid queries, write out, and copy to output folder for mapping target
valid_query_dict = {}
for target, val in query_dict.items():
    if val["syntax_check"] == "Passed":
        valid_query_dict[target] = val
final_query_dict = {
    "dataset_id": dataset_id,
    "transforms": valid_query_dict
}
query_dict_json = json.dumps(final_query_dict)
query_output_file = "transform_query_set.json"
with open(query_output_file, 'w') as outfile:
    outfile.write(query_dict_json)
destination_path = "ingest_pipeline/output/transformed/{}/{}/queries".format(mapping_target, dataset_id)
!gsutil cp $query_output_file $ws_bucket/$destination_path/ 2> stdout


# "T" Pipeline: Load Additional Transformed Tables to TDR

## Pipeline Run Variables

In [ ]:
# Run Variables
dataset_id_run_list = [
    #["dataset_id", Run (True/False)],
    ["1234", False],
    ["5678", False]
]
params["mapping_target"] = "anvil"
params["skip_transforms"] = False
params["transform_list_override"] = [] # Leave empty to run transforms for all files, otherwise populate with target table names 
params["skip_schema_extension"] = False
params["skip_ingests"] = False
params["ingest_list_override"] = [] # Leave empty to run ingests for all files, otherwise populate with target table names
params["skip_snapshot_creation"] = False
params["snapshot_readers_list"] = ["anvil_tdr_ingest@firecloud.org", "azul-anvil-prod@firecloud.org"] # Auth domain groups are also added as readers automatically
params["skip_data_validation"] = False

#-----------------------------------------------------------------------------------------------------------#

# Print variables
print("Pipeline run variables set:")
print("Profile ID: " + params["profile_id"])
print("Mapping Target: " + params["mapping_target"])
print("Datasets to run: ")
api_client = utils.refresh_tdr_api_client()
api_client.client_side_validation = False
datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
dataset_skip_list = []
for dataset in dataset_id_run_list:
    if dataset[1]:
        dataset_id = dataset[0]
        try:
            dataset_info = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION", "PROPERTIES"]).to_dict()
            dataset_name = dataset_info["name"]
            phs_id = dataset_info["phs_id"]
            consent_name = dataset_info["properties"]["consent_name"]
            auth_domains = dataset_info["properties"]["auth_domains"]
            src_workspaces = dataset_info["properties"]["source_workspaces"]
        except:
            dataset_name = ""
            dataset_skip_list.append(dataset_id)
        if dataset_name:
            dataset_id = dataset[0]
            print("- " + dataset_name + " ({})".format(dataset_id))
            print("\t- PHS ID = " + phs_id)
            print("\t- Consent Short Name = " + consent_name)
            print("\t- Auth Domains = " + str(auth_domains))
            print("\t- Source Workspaces = " + str(src_workspaces))
if dataset_skip_list:
    print("Datasets to skip (they either don't exist or aren't accessible to the current user): ")
    print("\t- " + "\n\t- ".join(dataset_skip_list)) 
print("Skip transforms? " + str(params["skip_transforms"]))
print("Skip schema extension? " + str(params["skip_schema_extension"]))
print("Skip ingests? " + str(params["skip_ingests"]))
print("Skip snapshot creation? " + str(params["skip_snapshot_creation"]))
print("Skip data validation? " + str(params["skip_data_validation"]))


## Pipeline Execution

In [ ]:
# Loop through and execute pipeline for listed workspaces
for dataset in dataset_id_run_list:
    if dataset[1]:
        dataset_id = dataset[0]
        try:
            dataset_info = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION", "PROPERTIES"]).to_dict()
            dataset_name = dataset_info["name"]
            phs_id = dataset_info["phs_id"]
            consent_name = dataset_info["properties"]["consent_name"]
            auth_domains = dataset_info["properties"]["auth_domains"]
            src_workspaces = dataset_info["properties"]["source_workspaces"]
        except:
            dataset_name = ""
        if dataset_name:
            params["dataset_id"] = dataset_id
            params["dataset_name"] = dataset_name
            params["phs_id"] = phs_id
            params["consent_name"] = consent_name
            params["auth_domains"] = auth_domains
            utils.run_t_pipeline(params)
        

# Utility Scripts
Uncomment sections as necessary to accomplish various miscellaneous tasks.

## Cloud Storage Clean-up

In [ ]:
# Set the name, mapping target, and target dataset_id of the source workspace whose table data files should be removed 
clean_staging_area_data = True
staging_area_name = ""
clean_transformed_data = True
mapping_target = "anvil"
dataset_id = "103c17df-ae7f-4f25-8441-a5eafaebdf01"

# gsutil commands to remove dataset table data and data file inventories where they live
if clean_staging_area_data:
    !gsutil -m rm -r $ws_bucket/ingest_pipeline/input/$workspace_name
    !gsutil -m rm -r $ws_bucket/ingest_pipeline/output/source/$workspace_name/table_data
if clean_transformed_data:
    !gsutil -m rm -r $ws_bucket/ingest_pipeline/output/transformed/$mapping_target/$dataset_id/table_data


## TDR Dataset and/or Snapshot Deletion

In [ ]:
# Setup Google creds and establish TDR clients
api_client = utils.refresh_tdr_api_client()
datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
snapshots_api = data_repo_client.SnapshotsApi(api_client=api_client)

# Function to delete a specific TDR Snapshot
def delete_snapshot(snapshot_id):
    print("Attempting to delete snapshot = {}".format(snapshot_id))
    try:
        delete_snapshot_result, job_id = utils.wait_for_tdr_job(snapshots_api.delete_snapshot(id=snapshot_id))
        print("Result: {}".format(delete_snapshot_result))
    except Exception as e:
        print("Error: {}".format(e))

# Function to delete a specific TDR Dataset
def delete_dataset(dataset_id):
    print("Attempting to delete dataset = {}".format(dataset_id))
    try:
        delete_dataset_result, job_id = utils.wait_for_tdr_job(datasets_api.delete_dataset(id=dataset_id))
        print("Result: {}".format(delete_dataset_result))
    except Exception as e:
        print("Error: {}".format(e))

# Function to delete a specific TDR Dataset and all of its Snapshots
def delete_dataset_and_all_snapshots(dataset_id):
    print("Attempting to delete dataset = {} and all associated snapshots".format(dataset_id))
    dataset_id_list = [dataset_id]
    # Delete snapshots
    snapshot_list = snapshots_api.enumerate_snapshots(dataset_ids=dataset_id_list)
    if snapshot_list.items:
        for snapshot in snapshot_list.items:
            snapshot_id = str(snapshot.id)
            delete_snapshot(snapshot_id)
    # Delete dataset
    delete_dataset(dataset_id)

# Delete a snapshot
# snapshot_id = "ce7f4b37-ce00-45d2-bc31-82eac2148d5f"
# delete_snapshot(snapshot_id)

# Delete a dataset
# dataset_id = "26512be2-9ac0-4852-ac82-85fdc32bffea"
# delete_dataset(dataset_id)

# Delete a dataset and all associated snapshots
dataset_id = "ee3cee29-241f-4f8d-9e73-31c15538c747"
delete_dataset_and_all_snapshots(dataset_id)


## Soft Deletion of TDR Dataset Records

In [ ]:
# Setup Google creds and establish TDR clients
api_client = utils.refresh_tdr_api_client()
datasets_api = data_repo_client.DatasetsApi(api_client=api_client)

# Function to delete rows from a dataset
def delete_datarepo_rows(dataset_id, table_name, datarepo_row_ids):
    print("Attempting to delete specified rows from {} for dataset {}".format(table_name, dataset_id))
    if datarepo_row_ids:
        data_deletion_payload = {
            "deleteType": "soft",
            "specType": "jsonArray",
            "tables": [{
              "tableName": table_name,
              "jsonArraySpec": {
                "rowIds": datarepo_row_ids
              }
            }]
        }
        try:
            data_deletion_result, job_id = utils.wait_for_tdr_job(datasets_api.apply_dataset_data_deletion(id=dataset_id, data_deletion_request=data_deletion_payload))
            print("Result: {}".format(data_deletion_result))
        except Exception as e:
            print("Error: {}".format(str(e)))
    else:
        print("No datarepo_row_ids specified for deletion.")

# Function to collect all datarepo rows for a particular table within a dataset
def collect_all_datarepo_rows(dataset_id, table_name):
    try:
        response = datasets_api.retrieve_dataset(id=dataset_id, include=["ACCESS_INFORMATION"]).to_dict()
        bq_project = response["access_information"]["big_query"]["project_id"]
        bq_schema = response["access_information"]["big_query"]["dataset_name"]
    except Exception as e:
        print("Error retrieving BQ project and schema: {}".format(str(e)))
    client = bigquery.Client()
    query = "SELECT datarepo_row_id FROM `{project}.{schema}.{table}`".format(project = bq_project, schema = bq_schema, table = table_name)
    try:
        query_job = client.query(query)
        results = [row["datarepo_row_id"] for row in query_job]
        return results
    except Exception as e:
        print("Error retrieving datarepo_row_id list: {}".format(str(e)))

# Specify which rows should be deleted from which dataset and table                                  
dataset_id = "1234"
table_name = "ws_table"
#datarepo_row_ids = collect_all_datarepo_rows(dataset_id, table_name) # Uncomment to pull all datarepo_row_ids in the specified table
datarepo_row_ids = []
delete_datarepo_rows(dataset_id, table_name, datarepo_row_ids)
